# Welcome to the Practical Exam :)
### <span style="color:purple"> **Please read the instructions, metadata and tips carefully in the practical exam PDF available on moodle!** </span>

**Also:**

-You can add as many cells as you like to answer the questions.

-You can make use of caching or persisting your RDDs or Dataframes, this may speed up performance.

**VERY IMPORTANT:** 

**- Don't forget to comment your code. Try to explain your reasoning to solve the exercises.**

**- Add your name to the .zip file submitted.**

**- Submit a .zip file. Files with other extensions will have a penalization.**

# PART A:

##  <span style="color:blue"> **0) Warmup (1 point):** </span>

#### a) Load the pages text file (pages.txt) into your DBFS:

In [0]:
#Open the pages text file and save it in the RDD
# When we put .collect() we are able to see the resulting below
links = sc.textFile('dbfs:/FileStore/tables/pages.txt')
links.collect()

Out[1]: ['A -> B',
 'A -> C',
 'A -> D',
 'B -> D',
 'B -> A',
 'C -> D',
 'D -> E',
 'E -> D']

#### b) Using the Page Rank algorithm, which is the most popular page? (0.5):
##### Perform all the necessary transformations to the text file and implement the Page Rank algorithm in order to answer this question

In [0]:
#just to see the actual values:
links.map(lambda tup : (tup[0], list(tup[1]))).collect()

Out[2]: [('A', [' ']),
 ('A', [' ']),
 ('A', [' ']),
 ('B', [' ']),
 ('B', [' ']),
 ('C', [' ']),
 ('D', [' ']),
 ('E', [' '])]

In [0]:
def computeContribs(neighbors, rank):
  for neighbor in neighbors:
    yield(neighbor, rank/len(neighbors))

In [0]:
pages_neighbors = links.map(lambda x: x.split()).map(lambda y: (y[0],y[1])).distinct().groupByKey().persist()
pages_neighbors.collect()

Out[13]: [('C', <pyspark.resultiterable.ResultIterable at 0x7fd3927586a0>),
 ('A', <pyspark.resultiterable.ResultIterable at 0x7fd3926403d0>),
 ('B', <pyspark.resultiterable.ResultIterable at 0x7fd392640d00>),
 ('D', <pyspark.resultiterable.ResultIterable at 0x7fd39273b5e0>),
 ('E', <pyspark.resultiterable.ResultIterable at 0x7fd392644a90>)]

In [0]:
#Starting the Page Ranking Algorithm
#First, we created ranks with a default value of rank 1 for all pages:
#Going through the links RDD will allow us to obtain all the keys, meaning that, all the pages - link[0]
ranks = pages_neighbors.map(lambda x: (x[0], 1.0))
ranks.collect()

Out[14]: [('C', 1.0), ('A', 1.0), ('B', 1.0), ('D', 1.0), ('E', 1.0)]

In [0]:
for i in range(10):
  contributions = links.join(ranks).flatMap(lambda x: computeContribs(x[1][0], x[1][1]))
  ranks = contributions.reduceByKey(lambda rank1, rank2: rank1+rank2).map(lambda x: (x[0], x[1]*0.85+0.15))
  print(ranks.collect())

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [0]:
#Creating a function that yields the contributions of each page to its neighbours
def getContribs(neighbours,rank):
#This function recieves two inputs. A list, neighbours and float, rank. The list corresponds to the neighbours of a page.
#The float corresponds to its rank.

#For each neighbour:
 for n in neighbours:

#I will return a tuple where the key is the neighbour and the value is the contribution it will receive.
#The contribution will be the rank of the page divided by the number of neighbours it has.
    yield(n, rank/len(neighbours))
  
#Note that the page that is giving the contibution is not mentioned in the result. The result is simply a key-value pair.
#Where the key is the neighbour and the value is how many "popularity points" it will recieve from the page.
#So in the end, when we call this function for each page, the result will be an RDD filled with tuples that have the new
#Popularity points each page (aka neighbour) will recieve... to continue we will need to sum them...

#outside the function

#We begin the interactions for the Page Rank algorithm

#Number of interactions:
i=20

for iteration in range(i):

  #merging ranks and links in order to compute the contributions that each page will recieve into an overall data RDD:
  #So now each record in data will be a tuple and have the following shape:
  #(pageName, ([neighbours], rank))

  data = links.join(links)
  
  #Getting the RDD containing all the contribution entries that each page will get.
  
  contribs = data.flatMap(lambda dataRecord : getContribs(dataRecord[1][0], dataRecord[1][1]))
  
  #Summing all the contributions for each page in order to update the rank:
  
  ranks= contribs.reduceByKey(lambda x, y : x + y)
  
 #The new rank is not just the sum of the contributions! It is this sum *0.85 + 0.15.
#So I want to keep the key the same way, but update the value according to the formula:

  ranks = ranks.map(lambda tup : (tup[0], tup[1] * 0.85 + 0.15))

#Outside of the for loop

ranks = ranks.sortByKey()

for value in ranks.collect():
  print(value)
  

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-1449010305568817> in <module>
     46 #Outside of the for loop
     47 
---> 48 ranks = ranks.sortByKey()
     49 
     50 for value in ranks.collect():

/databricks/spark/python/pyspark/rdd.py in sortByKey(self, ascending, numPartitions, keyfunc)
    779         # the key-space into bins such that the bins have roughly the same
    780         # number of (key, value) pairs falling into them
--> 781         rddSize = self.count()
    782         if not rddSize:
    783             return self  # empty RDD

/databricks/spark/python/pyspark/rdd.py in count(self)
   1267         3
   1268         """
-> 1269         return self.mapPartitions(lambda i: [sum(1 for _ in i)]).sum()
   1270 
   1271     def stats(self):

/databricks/spark/python/pyspark/rdd.py in sum(self)
   1256         6.0
   1257         """
-> 1258         return

#### C) Will ignoring C's contributions change these results? (0.5):
#### In other words, exclude C as a referencing page and repeat the algorithm. 

***Note that we do not intend to discard links that reference C, only the links where C is referencing other pages.***

##  <span style="color:blue"> **1) Data Load, Cleaning, and Feature Engineering (4 points):** </span>

#### a) Load the Online Retail file (onlineretail.csv) data into your DBFS:

In [0]:
#Importar ficheiro para o DBFS (arquivo do data Bricks)
#Mostrar ficheiros disponiveis no DBFS
#Importar csv com ","
#-> Inicio das aulas práticas

#### b) Prepare the data for analysis (0.5):

##### For instance, load the data to a spark DataFrame, or create the necessary table in SparkSQL.

In [0]:
# Vamos utilizar o Sparq SQL, é preciso importar a "library"
# Fazer copia do arquivo para tabela SQL
# visualisar tabela forma bonita (dispolay em vez de show)
# fazer sorts e procurar valores anómalos
#-> lab 5, pdf 10 until 24

#### c) There are missing values in the columns CustomerID and Description. Discard all records that have null values in such columns (0.5):

In [0]:
# Filtrar valores pretendidos
# drop rows with null values in any values
# -> Não encotrámos nas aulas, ver as práticas.

#### d) Discard all the records in which Quantity has a negative number (1):

In [0]:
# Filtrar valores pretendidos
# pode ser feito por filtro de não nulls
# -> Não encotrámos nas aulas, ver as práticas.# 
# Display

#### e) Create a new feature/column that corresponds to the amount spent (amount_spent) per record (1):
##### This can be done by computing amount_spent = quantity*unit_price.

In [0]:
#definir nova coluna
#Nova coluna resultado de uma operação de mutiplicar
#Display

#### f) Parse the invoice_date column and create one column for each of the following bits of information: year; month; day; hour. Then filter out all the invoices from 2011 (1):

In [0]:
# fazer uma função lambda com split 
# depois usar a função para criar as colunas


## <span style="color:blue"> **2) Data Exploration (10 points):** </span>
### Using the ***data that you cleaned*** in the previous exercises, answer the following questions:

#### a) What are the top 5 customers with the highest number of orders? (1)
##### Remember that the same invoice number means one order even if it appears in many rows!

*Tip: if you opt for a query you might need to include the count of the number of distinct InvoiceNo!*

In [0]:
# select columns cliente e Invoice Nº
# Agrupar o invoice com uma operação de count disntint por cliente
# Sort by number of orders
# Selecionar top 5
# selecionar paises com valor superior ao mais baixo
# -> exercicio 9 do lab 5; 

#### b) What are the top 5 customers with the highest money spent? (1)
##### Consider using the amount_spent column you created since it is an auxiliary column that multiplies the UnitPrice by the Quantity!

In [0]:
# Select cliente, nova coluna "amount_spent" criada no 1) 3)
# Agrupar o invoice com uma operação de sum 
# Sort by number of orders
# Selecionar top 5
# selecionar paises com valor superior ao mais baixo
# -> exercicio 9 do lab 5; 

#### c) What are the top 20 countries with the most orders? (1)
##### Same logic as in exercise 1, but instead of considering the customers we are considering the country.

In [0]:
# select columns paises e Invoice Nº
# Agrupar o invoice com uma operação de count disntint por cliente
# Sort by number of orders
# Selecionar top 20
# selecionar paises com valor superior ao mais baixo
# -> exercicio 9 do lab 5;
# no excel correr combinações para ver se o nº do invoice se pode repetir para paises

#### d) What are the top 5 products that are purchased the most amount of times? (1)
##### Note that by most amount of times, we do NOT consider the quantity of the order, only the fact that that item was purchased.

In [0]:
# Contar o nº de vezes que cada produto aparece no data set
#Colunas a usar >Stock code< ou o description
# group by count and count distinct

#### e)  What is the name (Description) of the most expensive item? (3)

In [0]:
#Fazer sort by price and filter all the items with the highest value detected

#### f) What are the top ten most popular words in the Description column? (3)

##### This is a word count exercise.
*Tip: you can use `.rdd` to convert a dataframe to an RDD. This will be an RDD of row objects. Make sure your top 5 words are actually words, you may have to remove empty strings!*

In [0]:
# we need to identify the most used word
# we need to split all the words at the descriptions
# count all the words 
# create a list with the counts
# sort and show top 10

# PART B:

## <span style="color:blue"> **Machine Learning (3 points):** </span>

#### a) Load the insurance dataset and perform the following operations: (0.5)
##### - Filter out (remove from the dataset) the region column.
##### - Encode the values in the smoker column in the following manner:
1. If the person is a smoker his/her value in this column should be 1
2. If the person is not a smoker his/her value in this column should be 0

#### b) Perform a multiple linear regression in an attempt to predict the health insurance charges based on the clients information. At the end report the coeficients, the RMSE and the r2 of the model: (2.5)

##### Make sure to consider any transformation necessary to the dataframe (e.g., deriving new variables, dropping existing ones, scale transformations , etc...) Justify all your decisions.

# PART C:

## <span style="color:blue"> **Show us what you got! (2 points):** </span>

<img src ='https://media.tenor.com/images/d8f45316e96078d93b36374474b67bf6/tenor.png'>

##### a) Using the cleaned e-commerce dataset from Part A, search for interesting variables that could be related to the countries' purchases. You should obtain the data regarding the countries' characteristics from the Macroeconomics sector of Pordata https://www.pordata.pt/en/Theme/Europe/Macroeconomics-32

###### You should limit your analysis to no more than six countries of your choice from the e-commerce dataset.

- The idea is that you find a way to associate some variables from Pordata and the purchases dataset. Ask yourselves the following questions: **What can explain/contribute to the fact that some countries buy more? or less? What can be related to the number of sales? Or the total value of sales?**... The questions are endless! Try to be creative and look for some associations!

###### You will be evaluated based on the logic used to solve this problem, the "investigation questions" you raise, the adequacy of your methodologies and your creativity.

##### MAKE SURE YOU COMMENT YOUR LOGIC AND "TELL YOUR STORY". WHAT QUESTIONS DID YOU RAISE? HOW DID YOU SEEK TO ANSWER THEM?

#### <span style="color:purple"> **Good luck and I hope you enjoyed doing this assignment :) !!!** </span>

<span style="color:red"> **Don't forget to submit your exam in a folder containing both your outputs PDF and your notebook. Once again, comment your code and don't forget to name the .zip file and notebook with your student numbers :) </span>